In [1]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

In [3]:
from torchvision import transforms, datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image
import matplotlib.pyplot as plt
import os

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
class BaseModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(32 * 100 * 100, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        out = torch.relu(self.conv1(x))
        out = self.pool(out)
        out = out.view(-1, 32 * 100 * 100)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)  
        return out

In [6]:
model = BaseModel().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

criterion = nn.BCEWithLogitsLoss()

In [7]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20481025


In [8]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [9]:
data_dir = '/kaggle/working/data'

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [10]:
from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir = '/kaggle/working/data/train',
    transform=train_transforms
)
test_dataset = HairDataset(
    data_dir = '/kaggle/working/data/test',
    transform=test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [11]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6446, Acc: 0.6392, Val Loss: 0.6039, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5448, Acc: 0.7079, Val Loss: 0.7690, Val Acc: 0.5821
Epoch 3/10, Loss: 0.5151, Acc: 0.7266, Val Loss: 0.6815, Val Acc: 0.6070
Epoch 4/10, Loss: 0.4696, Acc: 0.7541, Val Loss: 0.7208, Val Acc: 0.6517
Epoch 5/10, Loss: 0.4525, Acc: 0.7840, Val Loss: 0.6366, Val Acc: 0.6766
Epoch 6/10, Loss: 0.4034, Acc: 0.8077, Val Loss: 0.6472, Val Acc: 0.6915
Epoch 7/10, Loss: 0.3259, Acc: 0.8602, Val Loss: 0.7483, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2576, Acc: 0.8914, Val Loss: 0.8658, Val Acc: 0.6716
Epoch 9/10, Loss: 0.1988, Acc: 0.9313, Val Loss: 0.8272, Val Acc: 0.7015
Epoch 10/10, Loss: 0.1624, Acc: 0.9376, Val Loss: 2.4677, Val Acc: 0.5572


In [12]:
train_loss = np.array(history['loss'])
train_acc = np.array(history['acc'])
np.median(train_acc), np.std(train_loss)

(0.7958801498127341, 0.1497657787776018)

In [15]:
train_aug_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]),
])
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [16]:
from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir = '/kaggle/working/data/train',
    transform=train_aug_transforms
)
test_dataset = HairDataset(
    data_dir = '/kaggle/working/data/test',
    transform=test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [17]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.8915, Acc: 0.6092, Val Loss: 0.8120, Val Acc: 0.5821
Epoch 2/10, Loss: 0.5862, Acc: 0.6792, Val Loss: 0.6318, Val Acc: 0.6766
Epoch 3/10, Loss: 0.5567, Acc: 0.7029, Val Loss: 0.5745, Val Acc: 0.6965
Epoch 4/10, Loss: 0.5898, Acc: 0.6792, Val Loss: 0.6599, Val Acc: 0.6318
Epoch 5/10, Loss: 0.5540, Acc: 0.7091, Val Loss: 0.5487, Val Acc: 0.7164
Epoch 6/10, Loss: 0.5371, Acc: 0.7216, Val Loss: 0.5529, Val Acc: 0.7313
Epoch 7/10, Loss: 0.5495, Acc: 0.7104, Val Loss: 0.5645, Val Acc: 0.6965
Epoch 8/10, Loss: 0.5168, Acc: 0.7378, Val Loss: 0.6475, Val Acc: 0.6915
Epoch 9/10, Loss: 0.5094, Acc: 0.7453, Val Loss: 0.5187, Val Acc: 0.7562
Epoch 10/10, Loss: 0.4798, Acc: 0.7578, Val Loss: 0.5511, Val Acc: 0.7065


In [19]:
test_loss = np.array(history['val_loss'])
test_acc = np.array(history['val_acc'])
np.mean(test_loss), np.mean(test_acc[-5:])

(0.6061646786495227, 0.7164179104477613)